In [27]:

from __future__ import unicode_literals
from IPython.core.display import display, HTML
import json
from time import sleep

from bs4 import BeautifulSoup
from kafka import KafkaConsumer, KafkaProducer

import requests
# -*- coding: utf-8 -*-

def get_calls():
    url = 'https://finance.yahoo.com/quote/TSLA/options?straddle=false'
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find_all('table', class_ = 'calls W(100%) Pos(r) Bd(0) Pt(0) list-options')[0] # Grab the first table
    except Exception as ex:
        print('Exception in get_trades')
        print(str(ex))
    finally:
        return table

def get_puts():
    hotels = []
    url = 'https://finance.yahoo.com/quote/TSLA/options?straddle=false'
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find_all('table', class_ = 'puts W(100%) Pos(r) list-options')[0] # Grab the first table
    except Exception as ex:
        print('Exception in get_trades')
        print(str(ex))
    finally:
        return table
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
    'Pragma': 'no-cache'
    }

In [28]:
HTML(str(get_calls()))

Accessing list


Contract Name<!-- react-text: 57 --><!-- /react-text -->,Last Trade Date<!-- react-text: 60 --><!-- /react-text -->,Strike,Last Price<!-- react-text: 67 --><!-- /react-text -->,Bid<!-- react-text: 70 --><!-- /react-text -->,Ask<!-- react-text: 73 --><!-- /react-text -->,Change<!-- react-text: 76 --><!-- /react-text -->,% Change<!-- react-text: 79 --><!-- /react-text -->,Volume<!-- react-text: 82 --><!-- /react-text -->,Open Interest<!-- react-text: 85 --><!-- /react-text -->,Implied Volatility<!-- react-text: 88 --><!-- /react-text -->
TSLA210205C00200000,2021-01-29 2:38PM EST,200.00,585.78,639.15,640.75,0.00,-,2,1,433.59%
TSLA210205C00250000,2021-01-29 9:30AM EST,250.00,578.85,589.25,590.85,0.00,-,1,1,388.09%
TSLA210205C00300000,2021-01-29 2:02PM EST,300.00,493.60,539.25,540.75,0.00,-,14,17,324.02%
TSLA210205C00350000,2021-02-01 3:56PM EST,350.00,490.08,489.15,490.85,+38.58,+8.54%,11,11,277.73%
TSLA210205C00360000,2021-02-01 2:01PM EST,360.00,471.95,479.15,480.85,+7.15,+1.54%,1,2,269.34%
TSLA210205C00370000,2021-01-22 11:39AM EST,370.00,464.75,469.15,470.85,0.00,-,2,1,261.13%
TSLA210205C00380000,2021-01-29 3:26PM EST,380.00,421.55,459.25,460.80,0.00,-,5,13,256.45%
TSLA210205C00390000,2021-01-28 1:22PM EST,390.00,448.35,449.30,450.85,0.00,-,1,12,254.10%
TSLA210205C00400000,2021-01-28 1:22PM EST,400.00,438.35,439.05,440.75,0.00,-,2,42,220.70%
TSLA210205C00410000,2021-02-01 2:10PM EST,410.00,422.60,429.30,430.75,+47.40,+12.63%,6,5,233.40%


In [29]:
HTML(str(get_puts()))

Accessing list


Contract Name<!-- react-text: 3240 --><!-- /react-text -->,Last Trade Date<!-- react-text: 3243 --><!-- /react-text -->,Strike,Last Price<!-- react-text: 3250 --><!-- /react-text -->,Bid<!-- react-text: 3253 --><!-- /react-text -->,Ask<!-- react-text: 3256 --><!-- /react-text -->,Change<!-- react-text: 3259 --><!-- /react-text -->,% Change<!-- react-text: 3262 --><!-- /react-text -->,Volume<!-- react-text: 3265 --><!-- /react-text -->,Open Interest<!-- react-text: 3268 --><!-- /react-text -->,Implied Volatility<!-- react-text: 3271 --><!-- /react-text -->
TSLA210205P00200000,2021-02-01 3:39PM EST,200.00,0.01,0.00,0.01,-0.04,-80.00%,728,"1,924",337.50%
TSLA210205P00250000,2021-02-01 3:42PM EST,250.00,0.01,0.00,0.01,-0.06,-85.71%,239,"2,563",287.50%
TSLA210205P00300000,2021-02-01 3:47PM EST,300.00,0.02,0.01,0.02,-0.10,-83.33%,325,"3,982",262.50%
TSLA210205P00350000,2021-02-01 2:02PM EST,350.00,0.01,0.01,0.03,-0.23,-95.83%,693,552,228.13%
TSLA210205P00360000,2021-02-01 3:49PM EST,360.00,0.03,0.00,0.05,-0.22,-88.00%,26,379,225.00%
TSLA210205P00370000,2021-02-01 3:56PM EST,370.00,0.01,0.01,0.04,-0.32,-96.97%,38,112,217.97%
TSLA210205P00380000,2021-02-01 2:14PM EST,380.00,0.03,0.02,0.03,-0.50,-94.34%,30,314,210.94%
TSLA210205P00390000,2021-02-01 3:10PM EST,390.00,0.02,0.02,0.03,-0.39,-95.12%,112,253,204.69%
TSLA210205P00400000,2021-02-01 3:53PM EST,400.00,0.03,0.02,0.03,-0.45,-93.75%,444,"3,225",198.44%
TSLA210205P00410000,2021-02-01 1:23PM EST,410.00,0.05,0.01,0.07,-0.47,-90.38%,146,382,199.22%
